In [1]:
import os, json, uuid, requests
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from pymongo import MongoClient
from bson import json_util as bsonju

# --- config ---
DEFAULT_QR_URL = "https://faas-sgp1-18bc02ac.doserverless.co/api/v1/web/fn-86217a9f-1135-4904-b49a-fe070d4e10c7/forms/generate-qr-code"
CURRENT_QR_API_URL = os.getenv("GENERATE_QR_CODE_URL", DEFAULT_QR_URL)

# --- setup ---
load_dotenv()
client = MongoClient(os.getenv("MONGODB_URI"))
db = client[os.getenv("MONGODB_DATABASE")]
col = db["inventories"]

# --- input payload ---
payload = {
    "cashier": "John Smith",
    "cashier_employee_number": "10001",
    "recorder": "Daniel Miller",
    "recorder_employee_number": "10009",
}

def _now_ms():
    return int(datetime.now(timezone.utc).timestamp() * 1000)

def _fetch_new_current_qr():
    try:
        r = requests.get(CURRENT_QR_API_URL, timeout=10)
        r.raise_for_status()
        data = r.json()
        val = data.get("current_form_qr_code")
        if not isinstance(val, str) or not val.strip():
            raise ValueError(f"Bad QR response: {data}")
        return val.strip()
    except Exception as e:
        raise RuntimeError(f"Failed to fetch current_form_qr_code from API: {e}")

def _bump_date_str(d):
    try:
        dt = datetime.fromisoformat(str(d))
        return (dt + timedelta(days=1)).date().isoformat()
    except Exception:
        try:
            dt = datetime.strptime(str(d), "%Y-%m-%d")
            return (dt + timedelta(days=1)).date().isoformat()
        except Exception:
            return ""

def _map_items_for_new_doc(prev_items):
    """
    For each previous item:
      - previous_quantity = previous.current_quantity (or 0 if missing)
      - sold = 0, addstock = 0, current_quantity = 0
      - keep identifiers/attrs (item_number, barcode, type, name, price, unit)
      - assign a new item 'id' and 'created' timestamp
    """
    out = []
    now = {"$date": _now_ms()}
    for it in (prev_items or []):
        out.append({
            "item_number": it.get("item_number", ""),
            "id": str(uuid.uuid4()),
            "created": now,
            # omit "updated" for a fresh empty row; add later when edited
            "barcode": it.get("barcode", ""),
            "type": it.get("type", ""),
            "name": it.get("name", ""),
            "price": it.get("price", 0),
            "unit": it.get("unit", ""),
            "previous_quantity": it.get("current_quantity", 0) or 0,
            "addstock": 0,
            "sold": 0,
            "current_quantity": it.get("current_quantity", 0) or 0,
        })
    return out

def _new_empty_inventory(prev_doc: dict, curr_qr_code: str):
    return {
        "id": str(uuid.uuid4()),
        "previous_form_qr_code": prev_doc.get("current_form_qr_code", "") or "",
        "current_form_qr_code": curr_qr_code,
        "date": _bump_date_str(prev_doc.get("date", "")),
        "location": prev_doc.get("location", "") or "",
        "tin": prev_doc.get("tin", "") or "",
        "branch": prev_doc.get("branch", "") or "",
        "created": {"$date": _now_ms()},
        "items": _map_items_for_new_doc(prev_doc.get("items", [])),
        "cashier": "",
        "cashier_employee_number": "",
        "recorder": "",
        "recorder_employee_number": "",
        "is_empty": True,
    }

# --- find exactly one empty form ---
candidates = list(col.find({"is_empty": True}, {"_id": 1}).limit(2))
if len(candidates) != 1:
    print(json.dumps({"ok": False, "reason": "need_exactly_one_empty_form", "count": len(candidates)}, indent=2))
else:
    empty_doc = col.find_one({"_id": candidates[0]["_id"]})

    # finalize current empty doc
    patch = {**payload, "is_empty": False}
    upd = col.update_one({"_id": empty_doc["_id"], "is_empty": True}, {"$set": patch})

    if upd.matched_count != 1 or upd.modified_count != 1:
        print(json.dumps({
            "ok": False,
            "reason": "patch_failed_or_doc_not_empty_anymore",
            "matched": upd.matched_count,
            "modified": upd.modified_count
        }, indent=2, default=bsonju.default))
    else:
        # fetch new qr for next doc
        new_current_qr = _fetch_new_current_qr()

        # create next empty doc (items mirrored with reset counters)
        new_doc = _new_empty_inventory(prev_doc=empty_doc, curr_qr_code=new_current_qr)
        ins = col.insert_one(new_doc)

        out = {
            "ok": True,
            "finalized_document": {
                "id": empty_doc.get("id"),
                "was_is_empty": True,
                "now_is_empty": False,
                "patched_fields": list(payload.keys())
            },
            "new_empty_document": {
                "id": new_doc["id"],
                "mongo_id": str(ins.inserted_id),
                "previous_form_qr_code": new_doc["previous_form_qr_code"],
                "current_form_qr_code": new_doc["current_form_qr_code"],
                "date": new_doc["date"],
                "location": new_doc["location"],
                "tin": new_doc["tin"],
                "branch": new_doc["branch"],
                "is_empty": True,
                "created": new_doc["created"],
                "items_copied": len(new_doc["items"])
            }
        }
        print(json.dumps(out, indent=2, default=bsonju.default))


{
  "ok": true,
  "finalized_document": {
    "id": "5be664d1-12c9-4388-93ee-34bea7aba955",
    "was_is_empty": true,
    "now_is_empty": false,
    "patched_fields": [
      "cashier",
      "cashier_employee_number",
      "recorder",
      "recorder_employee_number"
    ]
  },
  "new_empty_document": {
    "id": "0f8c394f-0a84-47a7-a887-7d6bc9af826c",
    "mongo_id": "68eecfe86abf71cefbf289c4",
    "previous_form_qr_code": "69120825",
    "current_form_qr_code": "19124755",
    "date": "2025-10-06",
    "location": "JEF Gas Station \u2013 Sikatuna Branch",
    "tin": "123-456-789",
    "branch": "JEF Biosciences Fuel Division",
    "is_empty": true,
    "created": {
      "$date": 1760481256204
    },
    "items_copied": 8
  }
}
